# Comtrade - Data Prep
This notebook contains data preparation for UN Comtrade data from Harvard Dataverse. Objective is to generate a cleaned parquet file to store in our S3 analytics bucket.

Overall the dataset is fairly clean as the provider already performs various data cleaning operations to provide consistent trade data. There are 23,884 records in the `sitc_product_code` column with value `ZZ`. We will cast these as `99` so that we can cast the column as `int` and save the data in `parquet` format to enable faster loading and reduce storage size from 1.9GB to 250MB.

In [1]:
import json
import requests
import numpy as np
import pandas as pd

In [2]:
path = '/path/to/data'

In [3]:
# Read data as downloaded from source
df = pd.read_csv(filepath_or_buffer=f'{path}/country_partner_sitcproduct2digit_year.tab',
                 sep='\t',
                 dtype={
                     'location_id': np.int64,
                     'partner_id': np.int64,
                     'product_id': np.int64,
                     'year': np.int64,
                     'export_value': np.int64,
                     'import_value': np.int64,
                     'sitc_eci': np.float64,
                     'sitc_coi': np.float64,
                     'location_code': object,
                     'partner_code': object,
                     'sitc_product_code': object
})

In [4]:
# Identify unique product codes
df['sitc_product_code'].unique()

array(['33', '78', '74', '93', '03', '66', '04', '27', '52', '65', '69',
       '71', '72', '77', '82', '84', '87', '89', '09', '53', '62', '64',
       '67', '75', '05', '85', '11', '51', '76', '81', '07', '55', '56',
       '59', '79', '54', '88', '26', '68', '83', '58', '01', '02', '29',
       '42', '63', '34', '08', 'ZZ', '00', '06', '12', '22', '23', '24',
       '28', '41', '43', '57', '61', '73', '94', '97', '21', '25', '95',
       '35', '91', '32', '96'], dtype=object)

In [5]:
# Set code ZZ to np.nan
df.loc[df['sitc_product_code'] == 'ZZ', 'sitc_product_code'] = '99'

In [6]:
# Cast product codes as int
df['sitc_product_code'] = df['sitc_product_code'].astype(int)

In [7]:
# Drop columns calculated by Harvard Dataverse
df = df.drop(['sitc_eci', 'sitc_coi'], axis=1)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29524294 entries, 0 to 29524293
Data columns (total 9 columns):
location_id          int64
partner_id           int64
product_id           int64
year                 int64
export_value         int64
import_value         int64
location_code        object
partner_code         object
sitc_product_code    int64
dtypes: int64(7), object(2)
memory usage: 2.0+ GB


In [9]:
df.head()

,location_id,partner_id,product_id,year,export_value,import_value,location_code,partner_code,sitc_product_code
0,0,2,122,2010,52172104,0,ABW,AGO,33
1,1,2,122,2013,82953,0,AFG,AGO,33
2,5,2,122,2009,441,0,AND,AGO,33
3,5,2,122,2014,2776,0,AND,AGO,33
4,6,2,122,1964,1517614,0,ANT,AGO,33


In [10]:
df.to_parquet(f'{path}/comtrade.parquet')

This gives us the raw data. Now, we only need to load and save country and commodity mappings from UN Comtrade

In [11]:
mapping = {
    'reporter_areas': 'https://comtrade.un.org/data/cache/reporterAreas.json',
    'partner_areas': 'https://comtrade.un.org/data/cache/partnerAreas.json',
    'commodity_classification': 'https://comtrade.un.org/Data/cache/classificationHS.json'
}

In [12]:
# Get mapping as json and save to our data path
for k, v in mapping.items():
    r = requests.get(v)
    data = json.loads(r.text)['results'][1:]
    with open(f'{path}/{k}.json', 'w') as f:
        json.dump(data, f)